In [158]:
from xclib.data import data_utils
import numpy as np
import pandas as pd
from math import log2
import time
import matplotlib.pyplot as plt

# Reading the data-set

In [160]:
# Read sparse file
train_x = data_utils.read_sparse_file('ass3_parta_data/train_x.txt', force_header=True)
train_x = np.array(train_x.toarray(),dtype =int)
train_y = pd.read_csv('ass3_parta_data/train_y.txt', sep="\n", header=None).to_numpy()

test_x = data_utils.read_sparse_file('ass3_parta_data/test_x.txt', force_header=True)
test_x = np.array(test_x.toarray(),dtype =int)
test_y = pd.read_csv('ass3_parta_data/test_y.txt', sep="\n", header=None).to_numpy()

val_x = data_utils.read_sparse_file('ass3_parta_data/valid_x.txt', force_header=True)
val_x = np.array(val_x.toarray(),dtype =int)
val_y = pd.read_csv('ass3_parta_data/valid_y.txt', sep="\n", header=None).to_numpy()

/home/vivek/.local/lib/python3.6/site-packages/xclib-0.96-py3.6-linux-x86_64.egg/xclib/data/data_utils.py:173: UserWarning: Header mis-match from inferred shape!
  warnings.warn("Header mis-match from inferred shape!")


# Calculating Entropy

In [162]:
def entropy(data):
    """
    Calculates the uncertanity of data :
    sparse distibuted data low entropy
    mixed distibuted data high entropy
    """
    totaldata = data.shape[0]
    
    Totalpositive = np.sum(data[:,-1]==1)
    Totalnegative = np.sum(data[:,-1]==0)
    H = 0 
    
    # When only one class of data -> Pure
    if Totalpositive ==0 or Totalnegative ==0:
        return 0
    
    else:
        positive_ratio = Totalpositive/totaldata
        negative_ratio = Totalnegative/totaldata
        H = -positive_ratio * log2(positive_ratio) + (-negative_ratio * log2(negative_ratio))
        return H

# Finding Info-gain 

In [144]:
def info_gain(data,true_data_index,false_data_index,H):
    """
    Takes data, entropy of that data (H)
    as well as a col number to find info_gain 
    if we split on that col median
    """
    totalDataSize =data.shape[0]
    true_data = data[true_data_index]
    false_data = data[false_data_index]
    
    true_branch_ratio = true_data.shape[0]/totalDataSize
    false_branch_ratio = false_data.shape[0]/totalDataSize  

    if true_branch_ratio == 0 or false_branch_ratio ==0:
        return 0
    
    trueEntropy  = entropy(true_data)
    falseEntropy = entropy(false_data)
    
    I = H - ((true_branch_ratio * trueEntropy) + (false_branch_ratio * falseEntropy))
    return I

In [164]:
def best_split(data,H):
    
    best_info_gain = -1
    best_attr = -1
#     split_median = -1
    trueIndex = None
    falseIndex = None
    # consider all attributes except the last one i.e label *y*
    medianList  = np.median(data[:,:-1],axis =0)

    
    for colIndex, median in enumerate(medianList):
        
        true_data_index = np.where(data[:,colIndex] <= median)
        false_data_index = np.where(data[:,colIndex] > median)

        
        IG = info_gain(data,true_data_index,false_data_index,H)

        if IG > best_info_gain:
            best_info_gain = IG
            best_attr = colIndex
            split_median = median
            trueIndex = true_data_index
            falseIndex = false_data_index

    return best_attr, split_median, trueIndex,falseIndex, best_info_gain

# Decision Node Class

Stores necessary info at each node of the decison tree model.

In [168]:
class Decision_nodes:

    total_decision_nodes = 0
    depth = 0
    
    def __init__(self,depth):
        self.depth = depth
        self.left_child =None
        self.right_chid = None
        self.column = None
        self.split_median = None
        self.label = None
        self.parent = None
        

        self.val_x_index  = None
        
        
        self.train_acc = 0
        self.test_acc = 0
        self.val_acc = 0
        
        #----------Class Variable ---------------
        #counts number of nodes created each time a new node is created
        Decision_nodes.total_decision_nodes+=1
        
        if depth > Decision_nodes.depth:
            Decision_nodes.depth = depth
        #----------------------------------------

    def assignLabel(self,data):
        if sum(data[:,-1]==0) >= sum(data[:,-1]==1):
            self.label = 0
        else:
            self.label = 1
            
            
    @classmethod        
    def get_node_count(cls):
        return cls.total_decision_nodes
    @classmethod
    def reset_node_count(cls):
        cls.total_decision_nodes = 0

# Decision Tree Class

Contains Following Methods: <br>
calculate_acc : Calculate accuracy at each node while tree grows <br>
Build_tree : Creating tree recursively <br>
Predict : Finds Accuracy for a given data <br>
node_count : Total nodes in Tree *present* <br>
get_all_decesion_nodes : Returns List of all nodes preset in tree *Level Wise* <br>
pruning : Method to prune fully grown tree with Validation data

In [170]:
class Decision_tree:
    

    
    
    def __init__(self,node,train_y,test_y,val_y):
        
        self.root = node
        
        self.train_y_pred = [-1]*train_y.shape[0]
        self.test_y_pred  = [-1]*test_y.shape[0]
        self.val_y_pred = [-1]*val_y.shape[0]
        
        self.train_y = train_y
        self.test_y = test_y
        self.val_y = val_y
        
        self.match_train = 0
        self.match_test = 0
        self.match_val = 0
        
        self.train_accuracy = []
        self.test_accuracy = []
        self.val_accuracy = []
        
       

    def calculate_acc(self,label, Train_data, Test_data, Val_data):
#         label =  node.label
        
        new_match_train = 0
        for row in Train_data:
            index = row[-1]
            if label == self.train_y_pred[index] :
                continue
            elif label != self.train_y_pred[index]  and label == self.train_y[index]:
                self.train_y_pred[index] = label
                new_match_train+=1
            elif label != self.train_y_pred[index]  and label != self.train_y[index]:
                if self.train_y_pred[index] == -1:
                    self.train_y_pred[index] = label
                    continue
                else:
                    self.train_y_pred[index] = label
                    new_match_train-=1
                    
        final_match_train = self.match_train + new_match_train
        
        new_match_test = 0
        for row in Test_data:
            index = row[-1]
            if label == self.test_y_pred[index] :
                continue
            elif label != self.test_y_pred[index]  and label == self.test_y[index]:
                self.test_y_pred[index] = label
                new_match_test+=1
            elif label != self.test_y_pred[index]  and label != self.test_y[index]:
                if self.test_y_pred[index] == -1:
                    self.test_y_pred[index] = label
                    continue
                else:
                    self.test_y_pred[index] = label
                    new_match_test-=1
        final_match_test = self.match_test + new_match_test
        
        new_match_val = 0
        for row in Val_data:
            index = row[-1]
            if label == self.val_y_pred[index] :
                continue
            elif label != self.val_y_pred[index]  and label == self.val_y[index]:
                self.val_y_pred[index] = label
                new_match_val+=1
            elif label != self.val_y_pred[index]  and label != self.val_y[index]:
                if self.val_y_pred[index] == -1:
                    self.val_y_pred[index] = label
                    continue
                else:
                    self.val_y_pred[index] = label
                    new_match_val-=1
        final_match_val = self.match_val + new_match_val
        
        acc_train = final_match_train/train_y.shape[0]
        acc_test = final_match_test/test_y.shape[0]
        acc_val = final_match_val/val_y.shape[0]
        return final_match_train,final_match_test, final_match_val,acc_train, acc_test, acc_val
        
        
        
                

    def Build_tree(self,node, data, Train_data, Test_data, Val_data):


        H = entropy(data)
        node.assignLabel(data)

        node.val_x_index = Val_data[:,-1]
        best_attr, split_median, trueIndex, falseIndex, best_info_gain =  best_split(data,H)
        
        #---------------------------------------------------
        final_match_train,final_match_test, final_match_val,acc_train, acc_test, acc_val = self.calculate_acc(node.label,
                                                                                                              Train_data, 
                                                                                                              Test_data, 
                                                                                                              Val_data)

        self.match_train = final_match_train
        self.match_test = final_match_test
        self.match_val = final_match_val
        
        node.train_acc = final_match_train
        node.test_acc = final_match_test
        node.val_acc = final_match_val
        #---------------------------------------------------
        
        nc = Decision_nodes.get_node_count()

        if nc==1 or nc%88 == 1:
            print("node count ",nc)
            print("train_acc ",acc_train,"test_acc ",acc_test,"val_acc ",acc_val)
            self.train_accuracy.append((nc,acc_train))
            self.test_accuracy.append((nc,acc_test))
            self.val_accuracy.append((nc,acc_val))

        if H ==0 or best_attr == -1 or best_info_gain <= 0:
            return #LEAF

        true_branch = data[trueIndex]
        false_branch = data[falseIndex]

        if true_branch.shape[0] ==0 or false_branch.shape[0]==0:
            return #LEAF
        

        
        node.column  = best_attr
        node.split_median = split_median


        Train_data_left = Train_data[Train_data[:,best_attr] <= split_median]
        Train_data_right = Train_data[Train_data[:,best_attr] > split_median]

        Test_data_left = Test_data[Test_data[:,best_attr] <= split_median]
        Test_data_right = Test_data[Test_data[:,best_attr] > split_median]

        Val_data_left = Val_data[Val_data[:,best_attr] <= split_median]
        Val_data_right = Val_data[Val_data[:,best_attr] > split_median]

        left_node =  Decision_nodes(node.depth+1)
        right_node = Decision_nodes(node.depth+1)
        
        left_node.parent = node
        right_node.parent = node

        node.left_child = left_node
        self.Build_tree(left_node, true_branch, Train_data_left, Test_data_left, Val_data_left)
        node.right_chid = right_node
        self. Build_tree(right_node, false_branch, Train_data_right, Test_data_right, Val_data_right)

        return 
         
        


    def classify(self,x,node,mode):
     
            
        if node.left_child ==None and node.right_chid ==None:
            for row in x:
                index=row[-1]
                if mode =='val':
                    self.val_y_pred[index]  = node.label
                elif mode == 'train':
                    self.train_y_pred[index]  = node.label
                elif mode == 'test':
                    self.test_y_pred[index] = node.label
            return 

        col = node.column
        median = node.split_median

        left_x = x[x[:,col] <= median]
        right_x = x[x[:,col] > median]

        self.classify(left_x,node.left_child,mode)    
        self.classify(right_x,node.right_chid,mode)
        return 
    
        

        
    def predict(self,x,y):
        mode =''
        if y.shape[0] == len(self.val_y_pred):
            mode ='val'
            y_pred = self.val_y_pred
        elif y.shape[0] == len(self.train_y_pred):
            mode = 'train'
            y_pred = self.train_y_pred
        elif y.shape[0] == len(self.test_y_pred):
            mode = 'test'
            y_pred = self.test_y_pred
            
        match = 0
        x = np.append(x,np.arange(y.shape[0]).reshape((-1,1)),axis =1)
        self.classify(x, self.root, mode)
        for i in range(len(y_pred)):
            if y_pred[i] == y[i]:
                match += 1
        return match/y.shape[0]
    
    def node_count(self):
        count = 0
        queue = []
        queue.append(self.root)
        while(len(queue)!=0):
            count+=1
            node = queue.pop(0)
            if node.left_child:
                queue.append(node.left_child)
            if node.right_chid:
                queue.append(node.right_chid)
        return count
    
    def get_all_decesion_nodes(self):
        nodes_list = []
        
        q = []
        q.append(self.root)
        
        while(len(q)!=0):
            node = q.pop(0)
            if node.left_child:
                q.append(node.left_child)
            if node.right_chid:
                q.append(node.right_chid)
            if node.left_child!=None or node.right_chid!=None :# Decsion node
                nodes_list.append(node)
        return nodes_list
    
    def pruning(self):
        """
        pruning the decesion tree to remove
        overfitting.
        Pruning is done on validation data
        """

        
        train_acc_prune = []
        test_acc_prune = []
        val_acc_prune = []
        node_count_list = []
        
        node_count_list.append(self.node_count())
        train_acc_prune.append(self.predict(train_x,train_y))
        test_acc_prune.append(self.predict(test_x,test_y))
        val_acc_prune.append(self.predict(val_x,val_y))

        count  = 0
        print("entering wihile loop")

        while(True):
            count+=1
            #--------------------------------------------
            best_acc_gain = -1
            best_node_to_prune = None
            
            nodes_list = self.get_all_decesion_nodes()
            
            for node in nodes_list:
               
                y_pred = self.val_y_pred.copy()
                before_match = self.match_val
                before_acc = before_match/len(y_pred)
                after_match = before_match
                lc = node.left_child
                rc = node.right_chid

                if node.label != lc.label:
                    for index in lc.val_x_index:
                        if y_pred[index] == val_y[index]:
                            after_match-=1
                        if y_pred[index] != val_y[index]:
                            after_match+=1
                        y_pred[index] = node.label

                if node.label != rc.label:
                    for index in rc.val_x_index:
                        if y_pred[index] == val_y[index]:
                            after_match-=1
                        if y_pred[index] != val_y[index]:
                            after_match+=1
                        y_pred[index] = node.label

                after_acc = after_match/len(y_pred) 

                gain  = after_acc - before_acc
                if gain > best_acc_gain:
                    best_node_to_prune = node
                    best_acc_gain = gain
                    new_val_y_pred = y_pred
                    new_match_val = after_match
            
                    
            #---------------------------------------------    

            if best_acc_gain < 1e-20 or best_node_to_prune ==None:
                print("stopping here with gain ",best_acc_gain)
                break
            
            self.val_y_pred = new_val_y_pred
            self.match_val = new_match_val
            
            if best_node_to_prune.left_child:
                best_node_to_prune.left_child = None
            if best_node_to_prune.right_chid:
                best_node_to_prune.right_chid = None
            

                
            if count%50 ==1:   
                print("Number of Decsion Nodes",len(nodes_list))
                print("acc gain", best_acc_gain)
                node_count_list.append(self.node_count())
                train_acc_prune.append(self.predict(train_x,train_y))
                test_acc_prune.append(self.predict(test_x,test_y))
                val_acc_prune.append(self.predict(val_x,val_y))
            
        
        node_count_list.append(self.node_count())
        train_acc_prune.append(self.predict(train_x,train_y))
        test_acc_prune.append(self.predict(test_x,test_y))
        val_acc_prune.append(self.predict(val_x,val_y))
            
        print("train acc {} test acc {} val acc {}".format(self.predict(train_x,train_y),self.predict(test_x,test_y),self.predict(val_x,val_y)))
        tr, = plt.plot(node_count_list,train_acc_prune, label="Train Accuracy")
        te, = plt.plot(node_count_list,test_acc_prune, label ="Test Accuracy")
        vl, = plt.plot(node_count_list,val_acc_prune, label = "Validation Accuracy")
        plt.xlabel("Node Count")
        plt.ylabel("Accuracy")
        plt.title("Accuracy as we prune the nodes")
        plt.legend()
        plt.show()
        return tr,te,vl
       
            
 
       
          

# Creating indexed data for speeding the process

In [173]:
def get_indexed_data(train_x,train_y,test_x,test_y,val_x,val_y):
    Train_data = np.append(train_x,train_y.reshape((-1,1)),axis = 1)
    Test_data = np.append(test_x,test_y.reshape((-1,1)),axis = 1)
    Val_data = np.append(val_x,val_y.reshape((-1,1)),axis = 1)
    data = Train_data.copy()
    #index col
    Train_data = np.append(Train_data,np.arange(Train_data.shape[0]).reshape((-1,1)),axis =1)
    Test_data = np.append(Test_data,np.arange(Test_data.shape[0]).reshape((-1,1)),axis =1)
    Val_data = np.append(Val_data,np.arange(Val_data.shape[0]).reshape((-1,1)),axis =1)
    return Train_data, Test_data, Val_data,data

Train_data, Test_data, Val_data, data = get_indexed_data(train_x,train_y,test_x ,test_y ,val_x ,val_y )

# Part A
Growing the tree recusively. <br>
Calculating the accuracy at each node as the tree keeps growing. <br>
Plotting the graph of Nodes vs Accuracy

In [151]:
s= time.time()

root = Decision_nodes(0)

tree = Decision_tree(root,train_y,test_y ,val_y )

tree.Build_tree(root, data, Train_data, Test_data, Val_data)

print(time.time()-s)

node count  1
train_acc  0.5277610371949994 test_acc  0.5294144916786426 val_acc  0.522436491748563
node count  89
train_acc  0.5280700941078299 test_acc  0.5294608502155672 val_acc  0.5226682736881142
node count  177
train_acc  0.5291363404570951 test_acc  0.5301098697325113 val_acc  0.523178193955127
node count  265
train_acc  0.530897964860229 test_acc  0.5311297575448519 val_acc  0.5241053217133321
node count  353
train_acc  0.5310215876253612 test_acc  0.5310833990079273 val_acc  0.5241516781012423
node count  441
train_acc  0.5319487583638527 test_acc  0.531222474618701 val_acc  0.5243371036528833
node count  441
train_acc  0.5319642112094942 test_acc  0.5312688331556256 val_acc  0.5243371036528833
node count  441
train_acc  0.5319642112094942 test_acc  0.5311297575448519 val_acc  0.5243371036528833
node count  529
train_acc  0.5397369925671812 test_acc  0.5395206527281999 val_acc  0.5324031151492675
node count  617
train_acc  0.5411895600574845 test_acc  0.5406332576143897 val_a

node count  4049
train_acc  0.6109591581289695 test_acc  0.5877798896666822 val_acc  0.580103838308919
node count  4049
train_acc  0.6109591581289695 test_acc  0.5877798896666822 val_acc  0.580103838308919
node count  4049
train_acc  0.6109591581289695 test_acc  0.5877798896666822 val_acc  0.580103838308919
node count  4049
train_acc  0.6109591581289695 test_acc  0.5877798896666822 val_acc  0.580103838308919
node count  4049
train_acc  0.6110827808941016 test_acc  0.5879653238143804 val_acc  0.5801965510847394
node count  4137
train_acc  0.6114691020351398 test_acc  0.5879189652774558 val_acc  0.5805674021880215
node count  4137
train_acc  0.6114691020351398 test_acc  0.5879189652774558 val_acc  0.5805674021880215
node count  4225
train_acc  0.6118399703305364 test_acc  0.5880580408882295 val_acc  0.5807064713517522
node count  4313
train_acc  0.6120099516325932 test_acc  0.5879653238143804 val_acc  0.5807064713517522
node count  4401
train_acc  0.6123190085454236 test_acc  0.587965323

node count  7569
train_acc  0.6669293650425726 test_acc  0.6226415094339622 val_acc  0.615705544223994
node count  7569
train_acc  0.6669293650425726 test_acc  0.6226415094339622 val_acc  0.615705544223994
node count  7569
train_acc  0.6669293650425726 test_acc  0.6226415094339622 val_acc  0.615705544223994
node count  7657
train_acc  0.6674856674856675 test_acc  0.6228269435816606 val_acc  0.6154274058965326
node count  7657
train_acc  0.6674856674856675 test_acc  0.6228269435816606 val_acc  0.6154274058965326
node count  7657
train_acc  0.667501120331309 test_acc  0.6228269435816606 val_acc  0.6154274058965326
node count  7657
train_acc  0.6679956113918378 test_acc  0.623754114320152 val_acc  0.6165863155942889
node count  7745
train_acc  0.6696645187211225 test_acc  0.6248203606694173 val_acc  0.6179770072315965
node count  7833
train_acc  0.6724614837822385 test_acc  0.6263038338510036 val_acc  0.6200166882996477
node count  7921
train_acc  0.673064144762258 test_acc  0.62662834360

node count  11353
train_acc  0.710660918208088 test_acc  0.642065736405359 val_acc  0.6335527535694419
node count  11441
train_acc  0.7112017678055413 test_acc  0.6417875851838116 val_acc  0.6333673280178008
node count  11441
train_acc  0.7112017678055413 test_acc  0.6417875851838116 val_acc  0.6333673280178008
node count  11529
train_acc  0.7123761840742973 test_acc  0.6414630754253395 val_acc  0.6331819024661598
node count  11617
train_acc  0.7130097507455998 test_acc  0.6416485095730379 val_acc  0.6334136844057111
node count  11617
train_acc  0.7130097507455998 test_acc  0.6416485095730379 val_acc  0.6333673280178008
node count  11705
train_acc  0.7138905629471667 test_acc  0.6414630754253395 val_acc  0.6333209716298906
node count  11793
train_acc  0.7148795450682243 test_acc  0.6413239998145659 val_acc  0.6329501205266086
node count  11793
train_acc  0.7148795450682243 test_acc  0.6412776412776413 val_acc  0.6329501205266086
node count  11881
train_acc  0.7157603572697913 test_acc 

node count  14961
train_acc  0.8579883485543863 test_acc  0.7663993324370683 val_acc  0.7646949749675506
node count  15049
train_acc  0.8581737827020846 test_acc  0.7663066153632191 val_acc  0.7645559058038197
node count  15137
train_acc  0.8584364810779905 test_acc  0.76621389828937 val_acc  0.7645095494159095
node count  15225
train_acc  0.8586064623800472 test_acc  0.7661211812155209 val_acc  0.7644631930279993
node count  15225
train_acc  0.8586064623800472 test_acc  0.7661211812155209 val_acc  0.7644631930279993
node count  15225
train_acc  0.8586064623800472 test_acc  0.7661211812155209 val_acc  0.7644631930279993
node count  15225
train_acc  0.8586064623800472 test_acc  0.7661211812155209 val_acc  0.7644631930279993
node count  15225
train_acc  0.8586064623800472 test_acc  0.7661211812155209 val_acc  0.7644631930279993
node count  15313
train_acc  0.859008236366727 test_acc  0.7659357470678225 val_acc  0.76460226219173
node count  15313
train_acc  0.859008236366727 test_acc  0.7

node count  18481
train_acc  0.877350764143217 test_acc  0.7635714616846693 val_acc  0.7624698683478583
node count  18481
train_acc  0.877350764143217 test_acc  0.7635714616846693 val_acc  0.7624698683478583
node count  18481
train_acc  0.8773971226801416 test_acc  0.7635251031477447 val_acc  0.7624235119599481
node count  18569
train_acc  0.8779225194319534 test_acc  0.7633860275369709 val_acc  0.762052660856666
node count  18569
train_acc  0.877937972277595 test_acc  0.7633396690000464 val_acc  0.762052660856666
node count  18569
train_acc  0.8779534251232365 test_acc  0.7633396690000464 val_acc  0.762052660856666
node count  18569
train_acc  0.8779534251232365 test_acc  0.7633396690000464 val_acc  0.762052660856666
node count  18657
train_acc  0.8783861048011992 test_acc  0.7632933104631218 val_acc  0.762238086408307
node count  18745
train_acc  0.8790505771637848 test_acc  0.7631078763154235 val_acc  0.761867235305025
node count  18833
train_acc  0.8801631820499745 test_acc  0.7633

In [152]:
print("Train {} Test {} Val {}".format(tree.predict(train_x,train_y),tree.predict(test_x,test_y),tree.predict(val_x,val_y)))

Train 0.9043932440158855 Test 0.779843308145195 Val 0.7761913591692935


In [153]:
# import pickle 
# with open("tree",'wb') as f:
#     pickle.dump(tree,f)

# import pickle
# with open('tree','rb') as f:
#     tree = pickle.load(f)

In [154]:
# %matplotlib qt
def plot_node_vs_acc(tree):
    tr, = plt.plot([i[0] for i in tree.train_accuracy] ,[i[1] for i in tree.train_accuracy],label="train")
    te, = plt.plot([i[0] for i in tree.test_accuracy] ,[i[1] for i in tree.test_accuracy],label="test")
    vl, = plt.plot([i[0] for i in tree.val_accuracy] ,[i[1] for i in tree.val_accuracy],label="val")
    plt.xlabel("Number of nodes")
    plt.ylabel("Accuracies")
    plt.title("Train, Test and Val Accuracy while growing Decision Tree")
    plt.legend()
    plt.show()
    return tr,te,vl

In [155]:
plot_node_vs_acc(tree)

(<matplotlib.lines.Line2D at 0x7ff87f008cc0>,
 <matplotlib.lines.Line2D at 0x7ff87f076198>)

# Part B
Plotting the graph for nodes vs Accuracy in pruning the tree

In [157]:
s = time.time()
%matplotlib qt
tree.pruning()
print(time.time() -s )

entering wihile loop
Number of Decsion Nodes 9988
acc gain 0.00037085110328205495
Number of Decsion Nodes 9019
acc gain 0.00013906916373074285
Number of Decsion Nodes 8570
acc gain 9.271277582045823e-05
Number of Decsion Nodes 8088
acc gain 9.271277582056925e-05
Number of Decsion Nodes 7620
acc gain 9.271277582045823e-05
Number of Decsion Nodes 7014
acc gain 4.6356387910284624e-05
Number of Decsion Nodes 6375
acc gain 4.6356387910284624e-05
Number of Decsion Nodes 5922
acc gain 4.6356387910284624e-05
Number of Decsion Nodes 5616
acc gain 4.63563879101736e-05
Number of Decsion Nodes 5368
acc gain 4.6356387910284624e-05
stopping here with gain  0.0
train acc 0.8490411509279434 test acc 0.7931945667794724 val acc 0.8126738364546634
325.1944839954376


In [140]:
# def classify(row,node,depth):

#     if node.depth == depth:
#         return node.label
#     if node.left_child ==None and node.right_chid ==None:
#         return node.label

#     col = node.column
#     median = node.split_median
#     if row[col] <= median:
#         return classify(row, node.left_child,depth)
#     elif node.right_chid ==None:
#         return node.label
#     else:
#         return classify(row, node.right_chid,depth)

# def depthWiseAcc(x,y):
#     acc = []
#     for i in range(55):
#         r = tree.root
#         prediction = []
#         for row in x:
#             prediction.append(classify(row,r,i))
#         match=0
#         for i in range(len(prediction)):
#             if y[i]==prediction[i]:
#                 match+=1
#         acc.append(match/len(prediction))
        
#     return acc
# tr=depthWiseAcc(train_x,train_y)
# te=depthWiseAcc(test_x,test_y)
# vl = depthWiseAcc(val_x,val_y)

# %matplotlib qt
# plt.plot(np.arange(55),tr,label="train")
# plt.plot(np.arange(55),te,label="test")
# plt.plot(np.arange(55),vl,label="val")
# plt.xlabel("Levels in Decision Tree")
# plt.ylabel("Accuracies")
# plt.title("Train, Test and Val Accuracy while growing Decision Tree Level Wise")
# plt.legend()
# plt.show()

# Part C

In [179]:
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

param_test = {'n_estimators':np.arange(50,550,100),
              'max_features':np.arange(0.1,1.0,0.2),
              'min_samples_split': np.arange(2,12,2)}

In [180]:
from sklearn.metrics import  make_scorer
def oob_score(model,X,Y):
    return model.oob_score_
  
# score = make_scorer(oob_score)
forest =  RandomForestClassifier(n_jobs=-1,oob_score=True)

search = GridSearchCV(forest, param_grid=param_test, cv=3,verbose=3,scoring=oob_score)

In [184]:
search.fit(train_x, train_y.ravel())

__ n_estimators:  450 __ <br>
__ max_features:  0.1 __ <br>
__ min_samples_split:  10 __ <br>

# Part D

In [187]:
n_estimators = np.arange(50,550,100)
max_features = np.arange(0.1,1.0,0.2)
min_samples_split =  np.arange(2,12,2)

In [191]:
def n_estimators_plot():
    n_estimators_test_acc = []
    n_estimators_val_acc = []
    for i in n_estimators:
        model = RandomForestClassifier(n_jobs=-1,max_features = 0.1,min_samples_split = 10 ,n_estimators=i)
        model.fit(train_x,train_y)
        n_estimators_test_acc.append(model.score(test_x,test_y))
        n_estimators_val_acc.append(model.score(val_x,val_y))

    a, =plt.plot(np.arange(50,550,100),n_estimators_test_acc,label="test")
    b, =plt.plot(np.arange(50,550,100),n_estimators_val_acc,label="val")
    plt.xlabel('n_estimators',color='red')
    plt.ylabel('Accuracies',color='red')
    plt.title("Varying n_estimators")
    plt.legend()
    plt.show()
    return a,b

In [193]:
def max_features_plot():
    max_features_test_acc = []
    max_features_val_acc = []
    for i in max_features:
        model = RandomForestClassifier(n_jobs=-1,max_features = i,min_samples_split = 10 ,n_estimators=450)
        model.fit(train_x,train_y)
        max_features_test_acc.append(model.score(test_x,test_y))
        max_features_val_acc.append(model.score(val_x,val_y))

    a, = plt.plot(np.arange(0.1,1.0,0.2),max_features_test_acc,label="test")
    b, = plt.plot(np.arange(0.1,1.0,0.2),max_features_val_acc,label="val")
    plt.xlabel('max_features',color='red')
    plt.ylabel('Accuracies',color='red')
    plt.title("Varying max_features")
    plt.legend()
    plt.show()
    return a,b

In [194]:
def min_samples_split_plot():
    min_samples_split_test_acc = []
    min_samples_split_val_acc = []
    for i in min_samples_split:
        model = RandomForestClassifier(n_jobs=-1,max_features = 0.1,min_samples_split = i ,n_estimators=450)
        model.fit(train_x,train_y)
        min_samples_split_test_acc.append(model.score(test_x,test_y))
        min_samples_split_val_acc.append(model.score(val_x,val_y))



    a, = plt.plot(np.arange(2,12,2),min_samples_split_test_acc,label="test")
    b, = plt.plot(np.arange(2,12,2),min_samples_split_val_acc,label="val")
    plt.xlabel('min_samples_split',color='red')
    plt.ylabel('Accuracies',color='red')
    plt.title("Varying min_samples_split")
    plt.legend()
    plt.show()
    return a,b